# 导入必要的库

In [ ]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matlab.engine
import time
import scipy.io
import pandas as pd
import pickle
import scipy
from accProcess import *
sns.set_style('ticks')
sns.set_context("poster")
plt.rcParams['font.sans-serif'] = 'Arial'
# matlabeng = matlab.engine.start_matlab()   #启动matlab

# 指定模型和网络，加载数据

In [ ]:
model_path = 'inp_arg3_model'
result_path = 'basic_3inps_100class_logsa'
figure_path = 'result_figs'
fig_fmt = 'png'
if not os.path.exists(os.path.join(model_path, result_path, figure_path)):
    os.mkdir(os.path.join(model_path, result_path, figure_path))
with open(os.path.join(model_path, 'data_preprocess_100class_logsa.pkl'), 'rb') as file:
    dataset = pickle.loads(file.read())
performance = scipy.io.loadmat(os.path.join(model_path, result_path, 'performance.mat'))
results = scipy.io.loadmat(os.path.join(model_path, result_path, 'result.mat'))
print(performance.keys())
print(results.keys())
period = np.logspace(np.log10(0.01), np.log10(10), 200)

# Train、Valid或Test数据集全部台站结果可视化

## 指定训练集，获取这些台站的数据，并计算残差

In [ ]:
# 获取数据
dataset_type = 'train'
period_sec = [0.01, 0.1, 0.2, 0.5, 1.0, 2.0]
data = []
for i in range(len(dataset.train_data)):
    data.append([])
if dataset_type == 'train':
    for i in range(len(dataset.train_data)):
        data[i] = dataset.train_data[i]
    label = dataset.train_label
    pred = results['train_pred']
elif dataset_type == 'valid':
    for i in range(len(dataset.valid_data)):
        data[i] = dataset.valid_data[i]
    label = dataset.valid_label
    pred = results['valid_pred']
elif dataset_type == 'test':
    for i in range(len(dataset.test_data)):
        data[i] = dataset.test_data[i]
    label = dataset.test_label
    pred = results['test_pred']
for i in range(len(data)):
    data[i] = data[i] * dataset.data_std[i] + dataset.data_mean[i]
label = label * dataset.label_std + dataset.label_mean

# 计算残差
resi_type = 'abs'
if resi_type == 'abs':
    residual = pred - label
elif resi_type == 'rel':
    residual = (pred - label) / label
elif resi_type == 'log':
    residual = np.log(pred) - np.log(label)

data_scale = 'log'
if data_scale == 'log':
    label = np.log(10) * label
    pred = np.log(10) * pred
    data[0] = np.log(10) * data[0]
    residual = np.log(10) * residual

## 不同周期下预测和实际反应谱值对比图

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    r = np.corrcoef(label_points, pred_points)[0, 1]
    xmin = np.min(label_points)
    xmax = np.max([np.max(label_points), np.max(pred_points)])
    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=label_points, y=pred_points, label='T=%.2fs' % T)
    plt.plot([xmin, xmax], [xmin, xmax], 'k', linewidth=3)
    plt.text(xmin ** 0.9 * xmax ** 0.1, xmin ** 0.1 * xmax ** 0.9, 'r=%.1f%%' % (100 * r), size=28)
    plt.xlim([0.9 * xmin, 1.1 * xmax])
    plt.ylim([0.9 * xmin, 1.1 * xmax])
    if i >= 3:
        plt.xlabel('Target (gal)')
    if i in [0, 3]:
        plt.ylabel('Prediction (gal)')
    plt.xscale('log')
    plt.yscale('log')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_Sa_T_compare.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

## 绘制残差分布图

### 残差随周期的分布图

In [ ]:
N = len(period)
num_bar = 10
t_err, mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar), np.zeros(num_bar)
for i in range(num_bar):
    t_err[i] = period[int((i + 0.5) * N / num_bar)]
    mean_err[i] = np.mean(residual[:, int(i * N / num_bar) : int((i + 1) * N / num_bar), 0])
    std_err[i] = np.std(residual[:, int(i * N / num_bar) : int((i + 1) * N / num_bar), 0])

plt.figure(figsize=(8, 6))
sns.scatterplot(x=np.tile(period, residual.shape[0]), y=residual[..., 0].ravel(), s=50, color='silver')
plt.errorbar(t_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
plt.ylim([-0.6 * np.max(np.abs(residual[..., 0].ravel())), 0.6 * np.max(np.abs(residual[..., 0].ravel()))])
plt.xscale('log')
plt.xlabel('T (s)')
plt.ylabel('Residual')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_residual_T.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 残差随地表PGA的分布图

In [ ]:
# 加载加速度时程信息数据
event_msg = np.load('event_msg.npy', allow_pickle=True).item()
# 获取数据集中每个地震动对应的PGA数据
PGA_list = []
for i in range(label.shape[0]):
    if dataset_type == 'train':
        name = dataset.train_events[dataset.train_idx[i]]
    elif dataset_type == 'valid':
        name = dataset.valid_events[dataset.valid_idx[i]]
    elif dataset_type == 'test':
        name = dataset.test_events[dataset.test_idx[i]]
    PGA_list.append(981 * np.sqrt(event_msg[name][13] * event_msg[name][39]))

value_list = np.array(PGA_list)

# 将所有地震的PGA按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.min(value_list)
value_max = np.max(value_list)
num_bar = 10
value_step = (np.log(value_max) - np.log(value_min)) / num_bar
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.exp((np.arange(num_bar) + 0.5) * value_step + np.log(value_min))
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if np.log(value) >= np.log(value_min) + i * value_step and np.log(value) < np.log(value_min) + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随PGA的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err[:-1], mean_err[:-1], std_err[:-1], fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('PGA (gal)')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.xscale('log')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_residual_PGA.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 残差随$v_{s30}$的分布图

In [ ]:
# 加载vs30数据并获取数据集中每条地震动对应的Vs30值
vs30 = np.load('vs30.npy', allow_pickle=True).item()
vs30_list = []
for i in range(label.shape[0]):
    if dataset_type == 'train':
        name = dataset.train_events[dataset.train_idx[i]]
    elif dataset_type == 'valid':
        name = dataset.valid_events[dataset.valid_idx[i]]
    elif dataset_type == 'test':
        name = dataset.test_events[dataset.test_idx[i]]
    vs30_list.append(vs30[name[:6]])
value_list = np.array(vs30_list)

# 将所有地震对应的Vs30按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.min(value_list)
value_max = 1500
num_bar = 10
value_step = (value_max - value_min) / num_bar
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = value_min + (np.arange(num_bar) + 0.5) * value_step
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if value >= value_min + i * value_step and value < value_min + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随Vs30的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('$V_{s30}$ (m/s)')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.legend(loc='upper right')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_residual_vs30.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 残差随震级的分布图

In [ ]:
# 获取震级列表
value_list = data[2][:, 0, 0]

# 将所有地震对应的震级按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.round(np.min(value_list)) - 0.5
value_max = np.round(np.max(value_list)) + 0.5
value_step = 1
num_bar = int((value_max - value_min) / value_step)
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.arange(int(value_min + 0.5), int(value_max + 0.5))
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if value >= value_min + i * value_step and value < value_min + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随震级的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([value_min, value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('Mg.')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_residual_Mg.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 残差随震源深度的分布图

In [ ]:
# 获取震源深度列表
depth_list = data[2][:, 1, 0]
depth_list[depth_list < 1.0] = 1.0
value_list = depth_list

# 将所有地震对应的震源深度按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.min(value_list)
value_max = np.max(value_list)
num_bar = 10
value_step = (np.log(value_max) - np.log(value_min)) / num_bar
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.exp(np.log(value_min) + (np.arange(num_bar) + 0.5) * value_step)
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if np.log(value) >= np.log(value_min) + i * value_step and np.log(value) < np.log(value_min) + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随震源深度的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('Depth (km)')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.xscale('log')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_residual_Depth.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 残差随震中距的分布图

In [ ]:
# 获取震中距列表
value_list = data[2][:, 2, 0]

# 将所有地震对应的震中距按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.min(value_list)
value_max = np.max(value_list)
num_bar = 10
value_step = (np.log(value_max) - np.log(value_min)) / num_bar
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.exp(np.log(value_min) + (np.arange(num_bar) + 0.5) * value_step)
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if np.log(value) >= np.log(value_min) + i * value_step and np.log(value) < np.log(value_min) + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随震中距的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('R (km)')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.xscale('log')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_residual_Rup.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

## 绘制label和prediction的散点对比图

### 随周期的变化图

In [ ]:
ymin = 0
ymax = np.max([np.max(label[..., 0].ravel()), np.max(pred[..., 0].ravel())])
plt.figure(figsize=(8, 6))
sns.scatterplot(x=np.tile(period, label.shape[0]), y=label[..., 0].ravel(), s=20, label='Target')
sns.scatterplot(x=np.tile(period, label.shape[0]), y=pred[..., 0].ravel(), s=20, label='Prediction')
# plt.ylim([ymin, 0.2 * ymax])
plt.xscale('log')
plt.xlabel('T (s)')
plt.ylabel('Sa (gal)')
plt.legend()
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_Sa_scatter_T.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 随地表PGA的变化图

In [ ]:
# 加载加速度时程信息数据
event_msg = np.load('event_msg.npy', allow_pickle=True).item()
# 获取数据集中每个地震动对应的PGA数据
PGA_list = []
for i in range(label.shape[0]):
    if dataset_type == 'train':
        name = dataset.train_events[dataset.train_idx[i]]
    elif dataset_type == 'valid':
        name = dataset.valid_events[dataset.valid_idx[i]]
    elif dataset_type == 'test':
        name = dataset.test_events[dataset.test_idx[i]]
    PGA_list.append(981 * np.sqrt(event_msg[name][13] * event_msg[name][39]))
value_list = np.array(PGA_list)

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(value_min, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('PGA (gal)')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.xscale('log')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_Sa_scatter_PGA.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 随$v_{s30}$的变化图

In [ ]:
# 加载vs30数据并获取数据集中每条地震动对应的Vs30值
vs30 = np.load('vs30.npy', allow_pickle=True).item()
vs30_list = []
for i in range(label.shape[0]):
    if dataset_type == 'train':
        name = dataset.train_events[dataset.train_idx[i]]
    elif dataset_type == 'valid':
        name = dataset.valid_events[dataset.valid_idx[i]]
    elif dataset_type == 'test':
        name = dataset.test_events[dataset.test_idx[i]]
    vs30_list.append(vs30[name[:6]])
value_list = np.array(vs30_list)

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(0.8 * value_max, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('$V_{s30}$ (m/s)')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.legend(loc='upper right')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_Sa_scatter_vs30.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 随震级的变化图

In [ ]:
# 获取震级列表
value_list = data[2][:, 0, 0]

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(value_min, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('Mg.')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_Sa_scatter_Mg.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 随震源深度的变化图

In [ ]:
# 获取震源深度列表
value_list = data[2][:, 1, 0]
value_list[value_list < 1.0] = 1.0

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(value_min, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('Depth (km)')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.xscale('log')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_Sa_scatter_Depth.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

### 随震中距的变化图

In [ ]:
# 获取震中距列表
value_list = data[2][:, 2, 0]

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(value_min, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('Depth (km)')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.xscale('log')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, '%s_Sa_scatter_Rup.%s' % (dataset_type, fig_fmt)), dpi=300, bbox_inches='tight')

# 特定台站结果可视化

## 指定台站，获取这些台站的数据，并计算残差

In [ ]:
# 获取指定台站的data, label, pred数据
stations = ['IBRH13', 'IWTH08']
data, label, pred = [], [], []
events = []
for i in range(len(dataset.train_data)):
    data.append([])

count = 0
for i, idx in enumerate(dataset.train_idx):
    if dataset.train_events[idx][:6] in stations:
        count += 1
        events.append(dataset.train_events[idx])
        label.append(dataset.train_label[i, ...])
        pred.append(results['train_pred'][i, ...])
        for j in range(len(dataset.train_data)):
            data[j].append(dataset.train_data[j][i, ...])
print('Train dataset contains %d events' % count)

count = 0
for i, idx in enumerate(dataset.valid_idx):
    if dataset.valid_events[idx][:6] in stations:
        count += 1
        events.append(dataset.valid_events[idx])
        label.append(dataset.valid_label[i, ...])
        pred.append(results['valid_pred'][i, ...])
        for j in range(len(dataset.valid_data)):
            data[j].append(dataset.valid_data[j][i, ...])
print('valid dataset contains %d events' % count)

count = 0
for i, idx in enumerate(dataset.test_idx):
    if dataset.test_events[idx][:6] in stations:
        count += 1
        events.append(dataset.test_events[idx])
        label.append(dataset.test_label[i, ...])
        pred.append(results['test_pred'][i, ...])
        for j in range(len(dataset.test_data)):
            data[j].append(dataset.test_data[j][i, ...])
print('test dataset contains %d events' % count)
        
label = np.array(label)
label = label * dataset.label_std + dataset.label_mean
pred = np.array(pred)
for i in range(len(data)):
    data[i] = np.array(data[i])
    data[i] = data[i] * dataset.data_std[i] + dataset.data_mean[i]

# 计算残差
period_sec = [0.01, 0.1, 0.2, 0.5, 1.0, 2.0]
resi_type = 'abs'

if resi_type == 'abs':
    residual = pred - label
elif resi_type == 'rel':
    residual = (pred - label) / label
elif resi_type == 'log':
    residual = np.log(pred) - np.log(label)

data_scale = 'log'
if data_scale == 'log':
    label = np.log(10) * label
    pred = np.log(10) * pred
    data[0] = np.log(10) * data[0]
    residual = np.log(10) * residual

## 不同周期下预测和实际反应谱值对比图

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    r = np.corrcoef(label_points, pred_points)[0, 1]
    xmin = np.min(label_points)
    xmax = np.max([np.max(label_points), np.max(pred_points)])
    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=label_points, y=pred_points, label='T=%.2fs' % T)
    plt.plot([xmin, xmax], [xmin, xmax], 'k', linewidth=3)
    plt.text(xmin ** 0.9 * xmax ** 0.1, xmin ** 0.1 * xmax ** 0.9, 'r=%.1f%%' % (100 * r), size=28)
    plt.xlim([0.9 * xmin, 1.1 * xmax])
    plt.ylim([0.9 * xmin, 1.1 * xmax])
    if i >= 3:
        plt.xlabel('Target (gal)')
    if i in [0, 3]:
        plt.ylabel('Prediction (gal)')
    plt.xscale('log')
    plt.yscale('log')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_Sa_T_compare.%s' % fig_fmt), dpi=300, bbox_inches='tight')

## 绘制残差分布图

### 残差随周期的分布图

In [ ]:
N = len(period)
num_bar = 10
t_err, mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar), np.zeros(num_bar)
for i in range(num_bar):
    t_err[i] = period[int((i + 0.5) * N / num_bar)]
    mean_err[i] = np.mean(residual[:, int(i * N / num_bar) : int((i + 1) * N / num_bar), 0])
    std_err[i] = np.std(residual[:, int(i * N / num_bar) : int((i + 1) * N / num_bar), 0])

plt.figure(figsize=(8, 6))
sns.scatterplot(x=np.tile(period, residual.shape[0]), y=residual[..., 0].ravel(), s=50, color='silver')
plt.errorbar(t_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
plt.ylim([-0.6 * np.max(np.abs(residual[..., 0].ravel())), 0.6 * np.max(np.abs(residual[..., 0].ravel()))])
plt.xscale('log')
plt.xlabel('T (s)')
plt.ylabel('Residual')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_residual_T.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 残差随地表PGA的分布图

In [ ]:
# 加载加速度时程信息数据
event_msg = np.load('event_msg.npy', allow_pickle=True).item()
# 获取数据集中每个地震动对应的PGA数据
PGA_list = []
for i in range(label.shape[0]):
    PGA_list.append(981 * np.sqrt(event_msg[events[i]][13] * event_msg[events[i]][39]))
value_list = np.array(PGA_list)

# 将所有地震的PGA按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.min(value_list)
value_max = np.max(value_list)
num_bar = 10
value_step = (np.log(value_max) - np.log(value_min)) / num_bar
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.exp((np.arange(num_bar) + 0.5) * value_step + np.log(value_min))
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if np.log(value) >= np.log(value_min) + i * value_step and np.log(value) < np.log(value_min) + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随PGA的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err[:], mean_err[:], std_err[:], fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('PGA (gal)')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.xscale('log')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_residual_PGA.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 残差随震级的分布图

In [ ]:
# 获取震级列表
value_list = data[2][:, 0, 0]

# 将所有地震对应的震级按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.round(np.min(value_list)) - 0.5
value_max = np.round(np.max(value_list)) + 0.5
value_step = 1
num_bar = int((value_max - value_min) / value_step)
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.arange(int(value_min + 0.5), int(value_max + 0.5))
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if value >= value_min + i * value_step and value < value_min + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随震级的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([value_min, value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('Mg.')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_residual_Mg.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 残差随震源深度的分布图

In [ ]:
# 获取震源深度列表
value_list = data[2][:, 1, 0]
value_list[value_list < 1.0] = 1.0

# 将所有地震对应的震源深度按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.min(value_list)
value_max = np.max(value_list)
num_bar = 10
value_step = (np.log(value_max) - np.log(value_min)) / num_bar
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.exp(np.log(value_min) + (np.arange(num_bar) + 0.5) * value_step)
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if np.log(value) >= np.log(value_min) + i * value_step and np.log(value) < np.log(value_min) + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随震源深度的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('Depth (km)')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.xscale('log')
    plt.legend(loc='lower right')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_residual_Depth.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 残差随震中距的分布图

In [ ]:
# 获取震中距列表
value_list = data[2][:, 2, 0]

# 将所有地震对应的震中距按从小到大的顺序划分为10组，以便后面求解误差棒
value_min = np.min(value_list)
value_max = np.max(value_list)
num_bar = 10
value_step = (np.log(value_max) - np.log(value_min)) / num_bar
sort_idx = np.argsort(value_list)
value_list = value_list[sort_idx]
N = len(value_list)
idx_err = []
value_err = np.exp(np.log(value_min) + (np.arange(num_bar) + 0.5) * value_step)
for i in range(num_bar):
    idx = []
    for j, value in enumerate(value_list):
        if np.log(value) >= np.log(value_min) + i * value_step and np.log(value) < np.log(value_min) + (i + 1) * value_step:
            idx.append(j)
    idx_err.append(idx)

# 绘制残差随震中距的分布图和分组误差棒
plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    residual_points = residual[:, idx, 0]
    residual_points = residual_points[sort_idx]

    mean_err, std_err = np.zeros(num_bar), np.zeros(num_bar)
    for j in range(num_bar):
        mean_err[j] = np.mean(residual_points[idx_err[j]])
        std_err[j] = np.std(residual_points[idx_err[j]])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=residual_points, label='T=%.2fs' % T, s=50, color='silver')
    plt.errorbar(value_err, mean_err, std_err, fmt='s-', ecolor='k', ms=10, mfc='r', mec='k', color='k', elinewidth=2.5, capsize=5, capthick=3)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([-0.6 * np.max(np.abs(residual_points)), 0.6 * np.max(np.abs(residual_points))])
    if i >= 3:
        plt.xlabel('R (km)')
    if i in [0, 3]:
        plt.ylabel('Residual')
    plt.xscale('log')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_residual_Rup.%s' % fig_fmt), dpi=300, bbox_inches='tight')

## 绘制label和prediction的散点对比图

### 随周期的变化图

In [ ]:
ymin = 0
ymax = np.max([np.max(label[..., 0].ravel()), np.max(pred[..., 0].ravel())])
plt.figure(figsize=(8, 6))
sns.scatterplot(x=np.tile(period, label.shape[0]), y=label[..., 0].ravel(), s=20, label='Target')
sns.scatterplot(x=np.tile(period, label.shape[0]), y=pred[..., 0].ravel(), s=20, label='Prediction')
# plt.ylim([ymin, 0.2 * ymax])
plt.xscale('log')
plt.xlabel('T (s)')
plt.ylabel('Sa (gal)')
plt.legend()
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_Sa_scatter_T.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 随地表PGA的变化图

In [ ]:
# 加载加速度时程信息数据
event_msg = np.load('event_msg.npy', allow_pickle=True).item()
# 获取数据集中每个地震动对应的PGA数据
PGA_list = []
for i in range(label.shape[0]):
    if dataset_type == 'train':
        name = dataset.train_events[dataset.train_idx[i]]
    elif dataset_type == 'valid':
        name = dataset.valid_events[dataset.valid_idx[i]]
    elif dataset_type == 'test':
        name = dataset.test_events[dataset.test_idx[i]]
    PGA_list.append(981 * np.sqrt(event_msg[name][13] * event_msg[name][39]))
value_list = np.array(PGA_list)

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(value_min ** 0.3 * value_max ** 0.7, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('PGA (gal)')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.xscale('log')
    plt.legend(loc='upper right')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_Sa_scatter_PGA.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 随震级的变化图

In [ ]:
# 获取震级列表
value_list = data[2][:, 0, 0]

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(0.8 * value_max, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('Mg.')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.legend(loc='upper right')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_Sa_scatter_Mg.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 随震源深度的变化图

In [ ]:
# 获取震源深度列表
value_list = data[2][:, 1, 0]
value_list[value_list < 1.0] = 1.0

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(value_min, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('Depth (km)')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.xscale('log')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_Sa_scatter_Depth.%s' % fig_fmt), dpi=300, bbox_inches='tight')

### 随震中距的变化图

In [ ]:
# 获取震中距列表
value_list = data[2][:, 2, 0]

value_min = np.min(value_list)
value_max = np.max(value_list)

plt.figure(figsize=(18, 9))
plt.subplots_adjust(left=0,bottom=0,top=1,right=1,hspace=0.2,wspace=0.2)
for i, T in enumerate(period_sec):
    idx = np.argmin(np.abs(period - T))
    label_points = label[:, idx, 0]
    pred_points = pred[:, idx, 0]
    ymin = 0
    ymax = np.max([np.max(label_points), np.max(pred_points)])

    plt.subplot(2, 3, i + 1)
    sns.scatterplot(x=value_list, y=label_points, label='Target', s=50)
    sns.scatterplot(x=value_list, y=pred_points, label='Prediction', s=50)
    plt.text(value_min, 0.12 * ymax, 'T=%.2fs' % T, size=24)
    plt.xlim([0.8 * value_min, 1.1 * value_max])
    plt.ylim([ymin, 0.2 * ymax])
    if i >= 3:
        plt.xlabel('Depth (km)')
    if i in [0, 3]:
        plt.ylabel('Sa (gal)')
    plt.xscale('log')
    plt.legend(loc='upper left')
plt.savefig(os.path.join(model_path, result_path, figure_path, 'Stations_Sa_scatter_Rup.%s' % fig_fmt), dpi=300, bbox_inches='tight')